In [1]:
#first run ollama serve in the terminal
%pip install langchain_community -q
%pip install langchain_chroma -q
%pip install langchain_ollama -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_chroma import Chroma
import chromadb
os.environ['USER_AGENT'] = 'dd'
persist_directory = "./chroma_db"
load_from_db = False  # set to False to scrape again and re-embed

local_embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")

web_pages = ["https://docs.ncsa.illinois.edu/systems/delta/en/latest/user_guide/architecture.html", "https://www.sdsc.edu/systems/expanse/index.html"]

In [12]:
if not load_from_db:
    persistent_client = chromadb.PersistentClient(path = persist_directory)
    collection = persistent_client.get_or_create_collection("test_collection")
    vector_store_from_client = Chroma(
        client=persistent_client,
        collection_name="test_collection",
        embedding_function=local_embeddings,
    )
    
    # Load and split
    loader = WebBaseLoader(web_pages)
    docs = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200, chunk_overlap=100, add_start_index=True
    )
    all_splits = text_splitter.split_documents(docs)

    # Embed and save
    vector_store_from_client.add_documents(documents=all_splits)
    
else:
    # Load from saved vector DB
    print("loading vector store from db...")
    persistent_client = chromadb.PersistentClient(path = persist_directory)
    vector_store_from_client = Chroma(
            client=persistent_client,
            collection_name="test_collection",
            embedding_function=local_embeddings,
    )

In [ ]:
retriever = vector_store_from_client.as_retriever(search_type="similarity", search_kwargs={"k": 3})

question = "How many compute nodes at at NCSA Delta?"
retrieved_docs = retriever.invoke(question)
context = ' '.join([doc.page_content for doc in retrieved_docs])
llm = OllamaLLM(model="llama3:latest")
response = llm.invoke(f"""Answer the question concisely:
           Question: {question}.
           Context: {context}
""")

print(response)

In [ ]:
# Run this if you want to check the data in the db

import sqlite3

# Create a SQL connection to our SQLite database
conn = sqlite3.connect("chroma_db/chroma.sqlite3")
cursor = conn.cursor()

# List tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", cursor.fetchall())

# Explore one (example: collection)
cursor.execute("SELECT * FROM embedding_metadata;")
for row in cursor.fetchall():
    print(row)

conn.close()